<a href="https://colab.research.google.com/github/AlessandroMessori/ANN_Challenge_1/blob/main/Basic_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GOOGLE DRIVE ACCESS

In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [5]:
import os 

os.listdir(INPUT_PATH + "/dataset.zip (Unzipped Files)/training")

['Blueberry',
 'Apple',
 'Corn',
 'Cherry',
 'Grape',
 'Orange',
 'Peach',
 'Pepper',
 'Potato',
 'Raspberry',
 'Soybean',
 'Squash',
 'Strawberry',
 'Tomato']

In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)


2.7.0


## GLOBAL VARIABLES

In [4]:
INPUT_PATH = "/drive/MyDrive/Colab Notebooks"
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 100
SEED = 123

EPOCHS = 20
NUM_FOLDS = 10
INPUT_SHAPE = (256,256, 3)

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  INPUT_PATH + "/dataset.zip (Unzipped Files)/training",
  validation_split=0.2,
  subset="training",
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  INPUT_PATH + "/dataset.zip (Unzipped Files)/training",
  validation_split=0.2,
  subset="validation",
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 17728 files belonging to 14 classes.
Using 14183 files for training.
Found 17728 files belonging to 14 classes.
Using 3545 files for validation.


In [7]:
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(SEED)
    )(input_layer)

    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(SEED)
    )(pool1)

    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(SEED)
    )(pool2)

    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool3)
    
    flattening_layer = tfkl.Dropout(0.3, seed=SEED)(flattening_layer)

    classifier_layer = tfkl.Dense(units=128, name='Classifier', activation='relu')(flattening_layer)

    classifier_layer = tfkl.Dropout(0.3, seed=SEED)(classifier_layer)
    
    output_layer = tfkl.Dense(units=14, activation='softmax', name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
    # Return the model
    return model

In [8]:
images_dir = INPUT_PATH + "/dataset.zip (Unzipped Files)/training"

VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

save_dir = '/saved_models/'
fold_var = 1

kf = KFold(n_splits = 10)

idg = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

with tf.device('/device:GPU:0'):
	
  train_data_generator = idg.flow_from_directory(
                  directory=images_dir, target_size=(IMG_WIDTH, IMG_HEIGHT),
                  batch_size=BATCH_SIZE, seed=SEED, subset="training")
  
  valid_data_generator = idg.flow_from_directory(
                  directory=images_dir, target_size=(IMG_WIDTH, IMG_HEIGHT), 
                  batch_size=BATCH_SIZE, seed=SEED, subset="validation")
  
  #train_data_generator = tf.image.resize(train_data_generator, (64, 64))
  #valid_data_generator = tf.image.resize(valid_data_generator, (64, 64))
	
	# CREATE NEW MODEL
  model = build_model(INPUT_SHAPE)
	
	# CREATE CALLBACKS
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+str(0), 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
 
  callbacks_list = [checkpoint]
	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
  history = model.fit(train_data_generator,
			    epochs=EPOCHS,
			    callbacks=callbacks_list,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
  model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1
	

Found 14189 images belonging to 14 classes.
Found 3539 images belonging to 14 classes.
Epoch 1/20
142/142 [==============================] - ETA: 0s - loss: 1.4405 - accuracy: 0.5620 
Epoch 00001: val_accuracy improved from -inf to 0.68663, saving model to /saved_models/0


INFO:tensorflow:Assets written to: /saved_models/0/assets


INFO:tensorflow:Assets written to: /saved_models/0/assets


142/142 [==============================] - 2904s 20s/step - loss: 1.4405 - accuracy: 0.5620 - val_loss: 1.1034 - val_accuracy: 0.6866
Epoch 2/20
142/142 [==============================] - ETA: 0s - loss: 0.8112 - accuracy: 0.7441
Epoch 00002: val_accuracy improved from 0.68663 to 0.76632, saving model to /saved_models/0


INFO:tensorflow:Assets written to: /saved_models/0/assets


...

142/142 [==============================] - 277s 2s/step - loss: 0.1797 - accuracy: 0.9423 - val_loss: 0.3236 - val_accuracy: 0.9025
Epoch 15/20
142/142 [==============================] - ETA: 0s - loss: 0.1535 - accuracy: 0.9478
Epoch 00015: val_accuracy improved from 0.90251 to 0.92399, saving model to /saved_models/0


INFO:tensorflow:Assets written to: /saved_models/0/assets


INFO:tensorflow:Assets written to: /saved_models/0/assets


142/142 [==============================] - 279s 2s/step - loss: 0.1535 - accuracy: 0.9478 - val_loss: 0.2393 - val_accuracy: 0.9240
Epoch 16/20
142/142 [==============================] - ETA: 0s - loss: 0.1463 - accuracy: 0.9550
Epoch 00016: val_accuracy did not improve from 0.92399
142/142 [==============================] - 281s 2s/step - loss: 0.1463 - accuracy: 0.9550 - val_loss: 0.3036 - val_accuracy: 0.9101
Epoch 17/20
142/142 [==============================] - ETA: 0s - loss: 0.1399 - accuracy: 0.9531
Epoch 00017: val_accuracy did not improve from 0.92399
142/142 [==============================] - 273s 2s/step - loss: 0.1399 - accuracy: 0.9531 - val_loss: 0.3066 - val_accuracy: 0.9127
Epoch 18/20
142/142 [==============================] - ETA: 0s - loss: 0.1196 - accuracy: 0.9579
Epoch 00018: val_accuracy did not improve from 0.92399
142/142 [==============================] - 274s 2s/step - loss: 0.1196 - accuracy: 0.9579 - val_loss: 0.2928 - val_accuracy: 0.9152
Epoch 19/20
142/

OSError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')